In [1]:
import pandas as pd
import urllib
import os
import httplib2
import numpy as np
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import sys

sys.path.append('../src/data/')
import _init_paths
import imagedownloader
import pref_utils
from subprocess import Popen, PIPE
from os import environ

In [2]:
def source(script, update=1):
    pipe = Popen(". %s; env" % script, stdout=PIPE, shell=True)
    data = pipe.communicate()[0]

    env = dict((line.split("=", 1) for line in data.splitlines()))
    if update:
        environ.update(env)

    return env

In [3]:
# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
CLIENT_SECRET_FILE = '../client_secret.json'
APPLICATION_NAME = 'Test'


def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'sheets.googleapis.com-python-quickstart.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print 'Storing credentials to ' + credential_path 
    return credentials


"""Shows basic usage of the Sheets API.

ingrdients spreadsheet:
https://docs.google.com/spreadsheets/d/1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg/edit
"""
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

spreadsheetId = '1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg'
rangeName = 'All_in_one'
result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range=rangeName).execute()
values = result.get('values', [])

if not values:
    print 'No data found.'

scoodit_ingredients = pd.DataFrame(values[1:],columns=result.get('values')[0])
scoodit_ingredients = scoodit_ingredients.dropna(subset=['Categorie1'])
for col in scoodit_ingredients.columns:
    scoodit_ingredients[col] = scoodit_ingredients[col].str.lower().str.strip()
print 'list of scoodit ingredients from google doc'
scoodit_ingredients.head()

list of scoodit ingredients from google doc


,Group,Categorie1,Synonym1,Synonym2,Synonym3,Synonym4,Synonym5,wnid1,wnid2,wnid3
0,fruit,apple,,,,,,n07739125,None,None
1,fruit,apricot,,,,,,n07750872,None,None
2,fruit,banana,,,,,,n07753592,None,None
3,fruit,blackberry,,,,,,n07744811,None,None
4,fruit,blueberry,,,,,,n07743544,None,None


In [4]:
_ = source('../scoodit.env')

In [5]:
os.environ['IMAGENET_USERNAME']

'nsckir'

In [ ]:

downloader = imagedownloader.ImageNetDownloader()
username = os.environ['IMAGENET_USERNAME']
accessKey = os.environ['IMAGENET_ACCESSKEY']

userInfo = pref_utils.readUserInfo()
scoodit_wnids = {}
wnid_cols = scoodit_ingredients.columns[scoodit_ingredients.columns.str.contains('wnid')]
for ind, ing in scoodit_ingredients.iloc[:1].iterrows():
    for wnid in ing[wnid_cols].dropna():
        #download images
        downloader.downloadOriginalImages(wnid, username, accessKey)

        
        

Downloading: http://www.image-net.org/download/synset?wnid=n07739125&username=nsckir&accesskey=293780e006d78276e0080b5fc6033cd45010671f&release=latest&src=stanford Bytes: 139776000
